In [ ]:
import pandas as pd
survey = pd.read_excel('src_survey.xlsx')
onoff = pd.read_excel('src_OnOff.xlsx')

In [ ]:
#Объединяем два DataFrame по общему ключу (passport, unit)#
merged = pd.merge(survey, onoff, left_on=['passport', 'unit'], right_on=['PassportNumber', 'UnitNumber'])
print(merged)

In [ ]:
pd.to_numeric(merged['see'], errors='ignore')

In [ ]:
#Вычисляем количество ситуаций, когда прибор фактически выключен, а респондент утверждает, что включен
situation1 = merged[(merged['DeviceState'] == 'OFF') & (merged['tv state'] == 'включён')].shape[0]
print(situation1)

In [ ]:
#Вычисляем количество ситуаций, когда прибор фактически включен, а респондент утверждает, что выключен
situation2 = merged[(merged['DeviceState'] == 'ON') & (merged['tv state'] == 'выключен')].shape[0]
print(situation2)

In [ ]:
#Вычисляем общее количество ситуаций
total = merged.shape[0]
print(total)

In [ ]:
#Вычисляем проценты для каждой ситуации
percentage1 = round(situation1 / total * 100, 2)
percentage2 = round(situation2 / total * 100, 2)

In [ ]:
#Выводим результаты анализа
print(f'Количество ситуаций, когда прибор фактически выключен, а респондент утверждает, что включен: {situation1} ({percentage1}%)')
print(f'Количество ситуаций, когда прибор фактически включен, а респондент утверждает, что выключен: {situation2} ({percentage2}%)')

In [ ]:
#Выделяем ДХ с размером более 1 человека
households = merged[merged['Кол-во человек в семье 4+'] > 1]
print(households)

In [ ]:
#Выделяем ситуации 3.Выделить ДХ с размером более 1 человека, среди них ситуации, телевизор фактически включен, респондент по данным опроса находится в комнате с включенным телевизором,
# регистрации нет, а у других респондентов в этом ДХ регистрация на этом телевизоре есть. Необходимо сгруппировать результат в зависимости от размера ДХ.
situation3 = households[(households['DeviceState'] == 'ON') & (households['display'] == 'в комнате') & (households['see'] == 'нет регистрации')]
print(situation3)

In [ ]:
#Группируем результат по размеру ДХ
grouped = situation3.groupby('Кол-во человек в семье 4+').size()
#Выводим результаты анализа
print(f'Количество ситуаций по размеру ДХ:')
print(grouped)

Выводы:

•  Из результатов видно, что есть некоторое несоответствие между утверждениями респондентов и фактическим состоянием приборов. Это может быть связано с ошибками в опросе, неправильным пониманием вопросов или введением в заблуждение исследователей.

•  Есть ситуации, когда респондент находится в комнате с включенным телевизором, но не зарегистрирован на пиплметре, в то время как другие респонденты из того же ДХ зарегистрированы. Это может быть связано с техническими проблемами пиплметра, неправильным использованием кнопки или нежеланием респондента участвовать в исследовании.

•  Больше всего таких ситуаций происходит в ДХ с двумя членами. Это может быть объяснено тем, что в таких ДХ меньше контроля за поведением респондентов или больше вероятность того, что они будут находиться в одной комнате с одним телевизором.
